In [1]:
from typing import Tuple
import warnings

import neptune.new as neptune
import neptune.new.integrations.optuna as optuna_utils

import optuna
import numpy as np
import pandas as pd

from optuna.pruners import MedianPruner
from optuna.integration import SkoptSampler
from optuna.study import Study
from optuna.trial import FrozenTrial
from optuna.integration.xgboost import XGBoostPruningCallback 
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

warnings.filterwarnings("ignore")

In [2]:
def load_dataset(path: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train = pd.read_csv(path + "train.csv")
    test = pd.read_csv(path + "test.csv")

    drops = ["조식메뉴", "중식메뉴", "석식메뉴"]

    train = train.drop(drops, axis=1)
    test = test.drop(drops, axis=1)

    train["월"] = pd.DatetimeIndex(train["일자"]).month
    test["월"] = pd.DatetimeIndex(test["일자"]).month

    train["일"] = pd.DatetimeIndex(train["일자"]).day
    test["일"] = pd.DatetimeIndex(test["일자"]).day

    weekday = {"월": 1, "화": 2, "수": 3, "목": 4, "금": 5}

    train["요일"] = train["요일"].map(weekday)
    test["요일"] = test["요일"].map(weekday)

    train["식사가능자수"] = train["본사정원수"] - train["본사휴가자수"] - train["현본사소속재택근무자수"]
    test["식사가능자수"] = test["본사정원수"] - test["본사휴가자수"] - test["현본사소속재택근무자수"]

    train["중식참여율"] = train["중식계"] / train["식사가능자수"]
    train["석식참여율"] = train["석식계"] / train["식사가능자수"]

    features = ["월", "일", "요일", "식사가능자수", "본사출장자수", "본사시간외근무명령서승인건수"]
    labels = ["중식계", "석식계", "중식참여율", "석식참여율"]

    train = train[features + labels]
    test = test[features]

    # 요일을 석식 rank에 맞춰 mapping한 요일(석식) 칼럼 만들기.

    weekday_rank4dinner = {
        1: 1,
        2: 2,
        3: 5,
        4: 3,
        5: 4,
    }

    train["요일(석식)"] = train["요일"].map(weekday_rank4dinner)
    test["요일(석식)"] = test["요일"].map(weekday_rank4dinner)

    return train, test

In [3]:
train, test = load_dataset("../input/predict-meals/")


X_lunch = train[["월", "일", "요일", "식사가능자수", "본사출장자수", "본사시간외근무명령서승인건수"]]
y_lunch = train["중식계"]
X_test = test[["월", "일", "요일", "식사가능자수", "본사출장자수", "본사시간외근무명령서승인건수"]]

x_train, x_valid, y_train, y_valid = train_test_split(
    X_lunch, y_lunch, test_size=0.15, random_state=42
)

In [4]:
def objective(trial: FrozenTrial) -> float:
    param = {
        "lambda": trial.suggest_loguniform("lambda", 1e-03, 1e-01),
        "subsample": trial.suggest_float("subsample", 0.5, 1),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 300),
        "random_state": 42,
        "learning_rate": 0.02,
        "n_estimators": 10000,
        "eval_metric": "mae",
    }

    pruning_callback = XGBoostPruningCallback(trial, "validation_1-mae")
    model = XGBRegressor(**param)

    model.fit(
        x_train,
        y_train,
        eval_set=[(x_train, y_train), (x_valid, y_valid)],
        early_stopping_rounds=100,
        callbacks=[pruning_callback], # pruning과정을 callback할 수 있음.
        verbose=False,
    )

    preds = model.predict(x_valid)

    mae = mean_absolute_error(y_valid, preds)

    return mae

In [5]:
%%time
# run = neptune.init(
#     project="ds-wook/predict-meals",
#     # api_token="Anonymous",
# )

# # Create a NeptuneCallback for Optuna
# neptune_callback = optuna_utils.NeptuneCallback(
#     run,
#     plots_update_freq=1,  # create/log plots every 10 trials
#     log_plot_slice=False,  # do not create/log plot_slice
#     log_plot_contour=False,  # do not create/log plot_contour
# )

sampler = SkoptSampler(
    skopt_kwargs={
        "n_random_starts": 5,
        "acq_func": "EI",
        "acq_func_kwargs": {"xi": 0.02},
    }
)

study = optuna.create_study(
    study_name="optimization",
    direction="minimize",
    sampler=sampler,
    pruner=MedianPruner(n_warmup_steps=5),  # 튜닝시 과적합 될 경우 가지치기
)

study.optimize(
    objective,
    n_trials=200,
    # callbacks=[neptune_callback],
)
# run.stop()

[I 2021-08-11 09:52:01,415] A new study created in memory with name: optimization
[I 2021-08-11 09:52:02,531] Trial 0 finished with value: 94.21976837664019 and parameters: {'lambda': 0.022841550701823535, 'subsample': 0.8354551693646426, 'max_depth': 17, 'min_child_weight': 196}. Best is trial 0 with value: 94.21976837664019.
[I 2021-08-11 09:52:03,365] Trial 1 finished with value: 80.07076773459082 and parameters: {'lambda': 0.06099955342806396, 'subsample': 0.8645453099025653, 'max_depth': 10, 'min_child_weight': 123}. Best is trial 1 with value: 80.07076773459082.
[I 2021-08-11 09:52:03,993] Trial 2 finished with value: 75.61748255945702 and parameters: {'lambda': 0.009175013204679418, 'subsample': 0.7464646376126258, 'max_depth': 12, 'min_child_weight': 52}. Best is trial 2 with value: 75.61748255945702.
[I 2021-08-11 09:52:04,518] Trial 3 finished with value: 73.93327980673774 and parameters: {'lambda': 0.0013226331940643027, 'subsample': 0.7518212145901668, 'max_depth': 8, 'min_

[I 2021-08-11 09:52:38,615] Trial 76 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:39,057] Trial 77 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:39,483] Trial 78 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:39,925] Trial 79 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:40,399] Trial 80 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:40,796] Trial 81 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:41,255] Trial 82 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:41,732] Trial 83 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:42,369] Trial 84 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:42,808] Trial 85 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:43,197] Trial 86 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:43,660] Trial 87 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:52:44,077] Trial 88 pruned. Trial w

[I 2021-08-11 09:53:18,664] Trial 165 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:19,104] Trial 166 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:19,518] Trial 167 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:19,890] Trial 168 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:20,307] Trial 169 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:20,778] Trial 170 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:21,265] Trial 171 pruned. Trial was pruned at iteration 73.
[I 2021-08-11 09:53:21,647] Trial 172 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:22,034] Trial 173 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:22,449] Trial 174 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:22,871] Trial 175 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:23,235] Trial 176 pruned. Trial was pruned at iteration 44.
[I 2021-08-11 09:53:23,636] Trial 177 pr

CPU times: user 5min 29s, sys: 2min 28s, total: 7min 58s
Wall time: 1min 31s


In [6]:
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    '{key}': {value},")

Best trial:
  Value:  73.93327980673774
  Params: 
    'lambda': 0.0013226331940643027,
    'subsample': 0.7518212145901668,
    'max_depth': 8,
    'min_child_weight': 25,
